 1. Import Required Libraries

In [3]:
# TensorFlow and model building
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report

# For data handling and visualization
import os
import numpy as np
import matplotlib.pyplot as plt


2. Set Paths and Hyperparameters

In [ ]:
# Path to the dataset 
DATA_DIR = "./data"
IMG_SIZE = (244, 244)  #  increase to 224x224 for better accuracy
BATCH_SIZE = 32
EPOCHS = 20

Step 3: Data Preprocessing

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_path,
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data\\train'